# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pprint import pprint

from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing, ResNetProprioTunedRegression, ResNetProprioTunedSensorProcessing
from robot.al5d_position_controller import RobotPosition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
# run = "resnet50_128"
# run = "vgg19_256"
run = "resnet50_256"

exp = Config().get_experiment(experiment, run)

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
No system dependent experiment file
 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\resnet50_256_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/resnet50_256 successfully loaded


### Check the model for an encoding mode

Check whether we can load back the model. This loads back the model, not the sensor processing part. 

In [3]:
if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

if exp['loss'] == 'MSELoss':        
    criterion = nn.MSELoss()
elif exp['loss'] == 'L1Loss':
    criterion = nn.L1Loss()

modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
print(modelfile)
assert modelfile.exists()
model.load_state_dict(torch.load(modelfile))

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_propriotuned_cnn\resnet50_256\proprioception_mlp.pth


C:\Users\lboloni\AppData\Local\Temp\ipykernel_3312\265961778.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(modelfile))


<All keys matched successfully>

In [4]:
#
#  FIXME: this does not work on the resnet maybe the image format is not good
#

# Run the loaded model on all the images in a specific demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
# enc = model.to(device) # probably not need

inputlist = []
targetlist = []
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            z = model.encode(sensor_readings)
            break

print(f"The shape of the output is: {z.shape}")
print(f"One example z looks like this:\n{z}")


Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
The shape of the output is: torch.Size([1, 256])
One example z looks like this:
tensor([[-1.3334, -0.1345,  0.1154,  0.0816,  0.2175,  0.1926, -0.1940,  0.3807,
         -1.1284, -0.1088, -0.0457,  0.2652, -0.7553,  0.6069,  0.0619, -0.2752,
          0.8798,  0.2714, -2.0240,  0.1219, -0.3050, -0.0519, -0.0819,  0.2011,
          0.1430, -0.1386, -2.1422,  0.1663, -0.1422, -0.1606,  0.1884,  0.6170,
          0.4101, -0.8599, -0.1836,  0.8553, -0.0917,  0.2900, -0.2457, -0.7923,
          0.9542, -0.2464, -0.2466, -1.8020,  0.0466, -0.4877, -0.6147,  0.4019,
          0.7293, -0.6386,  0.0033,  0.2603,  0.2057,  0.0245,  0.3468, -0.9068,
          0.4963,  0.8772,  0.2232, -1.2107, -0.0134, -0.1352, -0.0656, -0.2550,
          0.1642,  0.0904, -0.0896,  0.0657, -0.4264, -0.5959, -0.1977,  0.0032,
         -0.1222, -0.5683, -1.4436, -0.0161, -0.7926, -1.9

### Show how to create and use a SensorProcessing object


In [5]:
pprint(exp)
# Create the sensor processing object

# exp = Config().get_experiment("sensorprocessing_propriotuned_cnn", run)

sp = None
if exp["model"] == "VGG19ProprioTunedRegression":
    sp = VGG19ProprioTunedSensorProcessing(exp, device)
if exp["model"] == "ResNetProprioTunedRegression":
    sp = ResNetProprioTunedSensorProcessing(exp, device)
if sp is None:
    raise Exception(f"Could not instantiate sp for {exp['model']}")

{'batch_size': 32,
 'data_dir': WindowsPath('c:/Users/lboloni/Documents/Code/_TempData/BerryPicker-experiments/sensorprocessing_propriotuned_cnn/resnet50_256'),
 'epochs': 100,
 'exp_run_sys_indep_file': WindowsPath('C:/Users/lboloni/Documents/Code/_Checkouts/BerryPicker/src/experiment_configs/sensorprocessing_propriotuned_cnn/resnet50_256.yaml'),
 'freeze_feature_extractor': True,
 'group_name': 'sensorprocessing_propriotuned_cnn',
 'latent_size': 256,
 'learning_rate': 0.002,
 'loss': 'MSELoss',
 'model': 'ResNetProprioTunedRegression',
 'model_name': 'VGG-19',
 'name': 'resnet-50-256',
 'output_size': 6,
 'proprio_step_1': 64,
 'proprio_step_2': 16,
 'proprioception_input_file': 'train_inputs.pt',
 'proprioception_mlp_model_file': 'proprioception_mlp.pth',
 'proprioception_target_file': 'train_targets.pt',
 'proprioception_test_input_file': 'test_inputs.pt',
 'proprioception_test_target_file': 'test_targets.pt',
 'proprioception_testing_task': 'proprio_sp_validation',
 'propriocepti

c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\..\sensorprocessing\sp_propriotuned_cnn.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sel

In [6]:
# Run the sensor processing package to process the images of a demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            sensor_readings = sensor_readings.to(device)
            z = sp.process(sensor_readings)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
[-1.3333772  -0.13453625  0.11544248  0.08157025  0.21745595  0.19263117
 -0.1939899   0.38074875 -1.1284058  -0.10883851 -0.04566706  0.26518828
 -0.7553371   0.60693103  0.06194327 -0.27515033  0.8797972   0.27140316
 -2.0240388   0.12191691 -0.304985   -0.05190174 -0.08188991  0.20107561
  0.14298907 -0.1386063  -2.1422384   0.16631186 -0.14223978 -0.16057381
  0.18842216  0.6169895   0.41014445 -0.85994744 -0.18359515  0.8552574
 -0.09166812  0.29004693 -0.2457342  -0.7922906   0.9541924  -0.24640244
 -0.24664524 -1.8020042   0.04662093 -0.4877024  -0.6146694   0.40185812
  0.7293241  -0.63863     0.0033437   0.26028037  0.2056734   0.0244971
  0.3467614  -0.90679455  0.49632847  0.87716794  0.22315413 -1.2106816
 -0.01343979 -0.1351868  -0.06555691 -0.25496584  0.16421692  0.09040243
 -0.0895507   0.0656828  -0.42644554 -0.59592056 -0.1976696   0.003

In [7]:
# Run the sensor processing inside the BCDemonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
        for i in range(bcd.trim_from, bcd.trim_to):
            #sensor_readings, _ = bcd.get_image(i)
            #sensor_readings = sensor_readings.to(device)
            #z = sp.process(sensor_readings)
            z = bcd.get_z(i)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
[-1.3333772  -0.13453625  0.11544248  0.08157025  0.21745595  0.19263117
 -0.1939899   0.38074875 -1.1284058  -0.10883851 -0.04566706  0.26518828
 -0.7553371   0.60693103  0.06194327 -0.27515033  0.8797972   0.27140316
 -2.0240388   0.12191691 -0.304985   -0.05190174 -0.08188991  0.20107561
  0.14298907 -0.1386063  -2.1422384   0.16631186 -0.14223978 -0.16057381
  0.18842216  0.6169895   0.41014445 -0.85994744 -0.18359515  0.8552574
 -0.09166812  0.29004693 -0.2457342  -0.7922906   0.9541924  -0.24640244
 -0.24664524 -1.8020042   0.04662093 -0.4877024  -0.6146694   0.40185812
  0.7293241  -0.63863     0.0033437   0.26028037  0.2056734   0.0244971
  0.3467614  -0.90679455  0.49632847  0.87716794  0.22315413 -1.2106816
 -0.01343979 -0.1351868  -0.06555691 -0.25496584  0.16421692  0.09040243
 -0.0895507   0.0656828  -0.42644554 -0.59592056 -0.1976696   0.003